# Numpy

## Лабораторная работа №1

Замечание: при решении данных задач не подразумевается использования циклов или генераторов Python, если в задании не сказано обратного. Решение должно опираться на использования функционала библиотеки `numpy`.

1. Файл `minutes_n_ingredients.csv` содержит информацию об идентификаторе рецепта, времени его выполнения в минутах и количестве необходимых ингредиентов. Считайте данные из этого файла в виде массива `numpy` типа `int32`, используя `np.loadtxt`. Выведите на экран первые 5 строк массива.

In [7]:
import numpy as np

data = np.loadtxt('minutes_n_ingredients.csv', delimiter=',', skiprows=1, dtype = np.int32)
print(data[0:5, :])

[[127244     60     16]
 [ 23891     25      7]
 [ 94746     10      6]
 [ 67660      5      6]
 [157911     60     14]]


2. Вычислите среднее значение, минимум, максимум и медиану по каждому из столбцов, кроме первого.

In [10]:
print(np.mean(data, axis = 0)[1:])
print([min(data[:, 1]), min(data[:, 2])])
print([max(data[:, 1]), max(data[:, 2])])
print(np.median(data, axis =0)[1:])

[2.16010017e+04 9.05528000e+00]
[0, 1]
[2147483647, 39]
[40.  9.]


3. Ограничьте сверху значения продолжительности выполнения рецепта значением квантиля $q_{0.75}$. 

In [46]:
data[data[:, 1] <= np.quantile(data[:, 1], 0.75)]

array([[127244,     60,     16],
       [ 23891,     25,      7],
       [ 94746,     10,      6],
       ...,
       [ 43407,     35,      7],
       [498432,     65,     15],
       [370915,      5,      4]])

4. Посчитайте, для скольких рецептов указана продолжительность, равная нулю. Замените для таких строк значение в данном столбце на 1.

In [45]:
print(np.count_nonzero(data[:, 1] == 0))
data[data == 0] = 1

479


5. Посчитайте, сколько уникальных рецептов находится в датасете.

In [170]:
len(np.unique(data[:, 0]))

100000

6. Сколько и каких различных значений кол-ва ингредиентов присутвует в рецептах из датасета?

In [29]:
ing = np.array(np.unique(data[:, 2])).T
print(f'Различных значений:{len(ing)} \n Значения:{ing}')

Различных значений:37 
 Значения:[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 37 39]


7. Создайте версию массива, содержащую информацию только о рецептах, состоящих не более чем из 5 ингредиентов.

In [178]:
recipes = data[data[:, 2] <= 5]
recipes

array([[446597,     15,      5],
       [204134,      5,      3],
       [ 25623,      6,      4],
       ...,
       [ 52088,     60,      5],
       [128811,     15,      4],
       [370915,      5,      4]])

8. Для каждого рецепта посчитайте, сколько в среднем ингредиентов приходится на одну минуту рецепта. Найдите максимальное значение этой величины для всего датасета

In [25]:
mean_recipe = data[:, 2] / data[:, 1]
print(f'Максимальное: {np.max(mean_recipe)} \n {mean_recipe}')

Максимальное: 24.0 
 [0.26666667 0.28       0.6        ... 0.23076923 0.8        0.1       ]


9. Вычислите среднее количество ингредиентов для топ-100 рецептов с наибольшей продолжительностью

In [220]:
mean_top = data[data[:, 1].argsort()[::-1]][0:101]
np.mean(mean_top)

7164107.623762377

10. Выберите случайным образом и выведите информацию о 10 различных рецептах

In [229]:
data[[np.random.choice(range(len(data)), 10)]]

array([[145616,     90,     14],
       [441689,     45,     11],
       [320131,      2,      3],
       [250313,     15,      8],
       [254001,     60,      8],
       [199389,     20,     24],
       [202846,     31,      4],
       [ 64656,     30,      8],
       [338146,     60,     12],
       [312164,     15,     10]])

11. Выведите процент рецептов, кол-во ингредиентов в которых меньше среднего.

In [305]:
fewer_mean = data[data[:, 2] < np.mean(data[:, 2])]
print(f'{(len(fewer_mean)/len(data))*100}%')

58.802%


12. Назовем "простым" такой рецепт, длительность выполнения которого не больше 20 минут и кол-во ингредиентов в котором не больше 5. Создайте версию датасета с дополнительным столбцом, значениями которого являются 1, если рецепт простой, и 0 в противном случае.

In [298]:
simple = np.where(((data[:, 1] <= 20) & (data[:, 2] <= 5)), 1, 0)
data = np.insert(data, 3, simple, axis= 1)
data

array([[127244,     60,     16,      0],
       [ 23891,     25,      7,      0],
       [ 94746,     10,      6,      0],
       ...,
       [498432,     65,     15,      0],
       [370915,      5,      4,      1],
       [ 81993,    140,     14,      0]])

13. Выведите процент "простых" рецептов в датасете

In [304]:
print(f'{(len(data[data[:, 3] == 1])/len(data))*100}%')

9.552%


14. Разделим рецепты на группы по следующему правилу. Назовем рецепты короткими, если их продолжительность составляет менее 10 минут; стандартными, если их продолжительность составляет более 10, но менее 20 минут; и длинными, если их продолжительность составляет не менее 20 минут. Создайте трехмерный массив, где нулевая ось отвечает за номер группы (короткий, стандартный или длинный рецепт), первая ось - за сам рецепт и вторая ось - за характеристики рецепта. Выберите максимальное количество рецептов из каждой группы таким образом, чтобы было возможно сформировать трехмерный массив. Выведите форму полученного массива.

In [2]:
small = data[data[:, 1] < 10]
standart = data[(data[:, 1] >= 10) & (data[:, 1] < 20)]
long = data[data[:, 1] >= 20]
num = min(small.shape[0], standart.shape[0], long.shape[0])
result = np.array([small[:num], standart[:num], long[:num]])
print(result)

[[[ 67660      5      6]
  [366174      7      9]
  [204134      5      3]
  ...
  [420725      5      3]
  [  4747      0      9]
  [370915      5      4]]

 [[ 94746     10      6]
  [ 33941     18      9]
  [446597     15      5]
  ...
  [  9831     15      7]
  [335859     12     14]
  [256812     10      3]]

 [[127244     60     16]
  [ 23891     25      7]
  [157911     60     14]
  ...
  [168901     25      7]
  [392339     35     13]
  [206732     45     10]]]
